<a href="https://colab.research.google.com/github/ipeirotis/autoencoders_census/blob/main/hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Autoencoders and Data Quality for Tabular Data**

This notebook searches the parameter space to create the best possible architecture for our autoencoder and saves the file into a Google Storage Bucket.

In [2]:
!rm -rf autoencoders_census
!git clone https://github.com/ipeirotis/autoencoders_census.git
%cd autoencoders_census

Cloning into 'autoencoders_census'...
remote: Enumerating objects: 349, done.
remote: Counting objects: 100% (226/226), done.
remote: Compressing objects: 100% (154/154), done.
remote: Total 349 (delta 136), reused 123 (delta 72), pack-reused 123
Receiving objects: 100% (349/349), 32.72 MiB | 9.12 MiB/s, done.
Resolving deltas: 100% (202/202), done.
/content/autoencoders_census


In [3]:
from google.colab import auth

# Login using the account that has access to the Google project
# in order to access the resources for the project
auth.authenticate_user()

In [4]:
!pip install -q import_ipynb keras-tuner
import import_ipynb
from pandas2vector import Table2Vector
from data_loader import DataLoader
from autoencoder import AutoencoderModel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.0 MB/s eta 0:00:00
importing Jupyter notebook from pandas2vector.ipynb
importing Jupyter notebook from data_loader.ipynb
importing Jupyter notebook from autoencoder.ipynb


# Data Source

The used data stems from the Youth Risk Behavior Surveillance System by CDC (Centers for Disease Control and Prevention). It includes data on a set of surveys that track behaviors that can lead to poor health in students grades 9 through 12. [Here](https://www.cdc.gov/healthyyouth/data/yrbs/data.htm) is the link to the dataset.

Note: The dataset is updated every two years. We use the 2017 version of the national high school YRBS dataset.

## Libraries

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Load Original Dataset

In [6]:
# Data Source
data_loader = DataLoader()


In [7]:
project_data, variable_types  = data_loader.load_2017()

In [8]:
project_data.shape

(14765, 98)

# Transforming the original data set

In [9]:
# Data Transformation
vectorizer = Table2Vector(variable_types)


# Without indicator variables for missingness
vectorized_df = vectorizer.vectorize_table(project_data)

In [10]:
vectorized_df.shape

(14765, 607)

In [11]:
vectorizer.tabularize_vector(vectorized_df).shape

(14765, 98)

# Autoencoder training

## Hyperparameter search and model training

The training of an autoencoder model while effectively handling missing data with mask variables.

In [12]:
project_data.describe().T

,count,unique,top,freq
age,14765,8,16 years old,3688
sex,14765,3,Female,7526
grade,14765,5,9th,3921
Hispanic_or_Latino,14765,5,White,6261
race,14765,8,White,6261
...,...,...,...,...
weight_cat,14765,2,normal,14428
stheight_cat,14765,4,normal,13127
stweight_cat,14765,3,normal,12955
bmi_cat,14765,4,normal,13037


In [13]:
attr_cardinalities = list(project_data.describe().T['unique'].values)
print(attr_cardinalities)

[8, 3, 5, 5, 8, 3, 3, 5, 5, 5, 4, 5, 4, 6, 6, 7, 9, 6, 6, 6, 9, 9, 9, 3, 7, 7, 3, 3, 3, 3, 3, 6, 4, 3, 8, 8, 3, 8, 8, 8, 8, 8, 9, 9, 8, 8, 7, 7, 7, 7, 7, 7, 7, 7, 4, 3, 3, 9, 8, 9, 4, 4, 9, 6, 5, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 8, 8, 7, 5, 4, 6, 4, 8, 8, 7, 8, 8, 4, 9, 7, 7, 3, 5, 2, 4, 3, 4, 2]


In [14]:
# Instantiate the class
model = AutoencoderModel(attr_cardinalities)
# Preprocess the data
X_train, X_test = model.split_train_test(vectorized_df)

# Define the search space

# how many times to run with the same configuration: offers better statistical significance
model.EXECUTIONS_PER_TRIAL=1

# how many runs over the training data: the more the better accuracy but takes time
model.EPOCHS=3

# how many different parameter configurations to try
model.MAX_TRIALS=10



In [15]:
# Define the tuner
tuner = model.define_tuner()

# Perform hyperparameter search
tuner.search(X_train, X_train, epochs=model.EPOCHS,
             batch_size=64,
             validation_data=(X_test, X_test))

# Get the best hyperparameters and build the final model
best_hps = tuner.get_best_hyperparameters()[0]
final_model = model.build_autoencoder(best_hps)

Trial 10 Complete [00h 01m 39s]
val_loss: 15.402154922485352

Best val_loss So Far: 0.5263448357582092
Total elapsed time: 00h 16m 25s


In [ ]:
best_hps.values

In [ ]:
# Train the final model
history = final_model.fit(X_train, X_train,
                           epochs=100*model.EPOCHS,
                           verbose=1,
                           validation_data=(X_test, X_test))

Visualize the training and validation loss values over epochs.

In [ ]:
def model_analysis(train_loss, val_loss):
    epochs = range(1, len(train_loss) + 1)

    plt.figure(figsize=(8, 6))
    plt.title("Cross categorical entropy loss")
    sns.lineplot(x=epochs[1:], y=train_loss[1:], label="Train", linewidth=3)
    sns.lineplot(x=epochs[1:], y=val_loss[1:], label="Validation", linewidth=3)
    plt.xlabel("Epochs")

    plt.legend()
    plt.show()

    print(f"Training = {(train_loss[-1])}")
    print(f"Validation = {(val_loss[-1])}")

model_analysis(history.history['loss'], history.history['val_loss'])

In [ ]:
import gcsfs
# Create a GCS filesystem object using your project ID
fs = gcsfs.GCSFileSystem(project='autoencoder_census')

# Save the model locally
model_filename = "best_parameters_autoencoder.keras"
final_model.save(model_filename)



# Define the path to the file on GCS
gcs_model_path = f"gs://autoencoder_census_models/{model_filename}"

# Open the local file in binary mode and upload its content to GCS
with open(model_filename, 'rb') as local_file:
    with fs.open(gcs_model_path, 'wb') as gcs_file:
        gcs_file.write(local_file.read())


In [ ]:


# Store the hyperparameters and evaluation metrics in a dictionary
hyperparameters_dict = {"params": best_hps.values,
                        "loss": history.history['loss'][-1],
                        "val_loss": history.history['val_loss'][-1]}

hyperparameters_dict